# Import dependencies

In [1]:
import os
import re
import math
import pickle
import numpy as np
import pandas as pd
import importlib

import spacy

from utilities import _support
from utilities._support import ig_profiles

from nltk.tokenize import word_tokenize, sent_tokenize

importlib.reload(_support)

Backing off send_request(...) for 2.1s (requests.exceptions.ConnectionError: HTTPSConnectionPool(host='api.segment.io', port=443): Max retries exceeded with url: /v1/batch (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000155BC7D45E0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')))
Backing off send_request(...) for 5.0s (requests.exceptions.ConnectionError: HTTPSConnectionPool(host='api.segment.io', port=443): Max retries exceeded with url: /v1/batch (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000155C16737C0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')))
Backing off send_request(...) for 11.1s (requests.exceptions.ConnectionError: HTTPSConnectionPool(host='api.segment.io', port=443): Max retries exceeded with url: /v1/batch (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000155C1686070>: Failed to establish a new connection: [

<module 'utilities._support' from 'c:\\Users\\tommaso\\Documents\\_Projects\\IGScraper\\utilities\\_support.py'>

# Load data

In [2]:
with open('utilities/lookup_users.pkl', 'rb') as f:
    full_lookup = pickle.load(f)

df = pd.read_json('gitshared_csv/matteosalviniofficial/matteosalviniofficial_comments_20230316_0-1100_preprocessed.json')

Backing off send_request(...) for 5.5s (requests.exceptions.ConnectionError: HTTPSConnectionPool(host='api.segment.io', port=443): Max retries exceeded with url: /v1/batch (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000155D28A65B0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')))


In [3]:
df

,post_id,user,comment,comment_datetime,comment_likes,comment_prep,comment_token,comment_lemma,comment_postag
0,Ck9MRyyKIT5,user_0000027564,"Tutti i cittadini dovrebbero saperlo praticare. Si salverebbero migliaia di vita ogni anno. Questa è civiltà, questa è la società che si evolve. Non con i bagni per chi non é sicuro del proprio genere...",2022-11-14T21:29:33.000Z,29,tutti i cittadini dovrebbero saperlo praticare. si salverebbero migliaia di vita ogni anno. questa è civiltà questa è la società che si evolve. non con i bagni per chi non é sicuro del proprio genere.,"[[tutti, i, cittadini, dovrebbero, saperlo, praticare], [si, salverebbero, migliaia, di, vita, ogni, anno], [questa, è, civiltà, questa, è, la, società, che, si, evolve], [non, con, i, bagni, per, chi, non, é, sicuro, del, proprio, genere]]","[[tutto, il, cittadino, dovere, saperlo, praticare], [si, salvare, migliaio, di, vita, ogni, anno], [questo, essere, civiltà, questo, essere, la, società, che, si, evolvere], [non, con, il, bagno, per, chi, non, é, sicuro, del, proprio, genera]]","[[T, RD, S, VM, V_PC, V], [PC, V, S, E, S, DI, S], [PD, V, S, PD, V, RD, S, PR, PC, V], [BN, E, RD, S, E, PR, BN, V, A, E_RD, AP, S]]"
1,Ck9MRyyKIT5,user_0000019631,"La Professionalità e tempestività di questa ragazza sono encomiabili! Bisognerebbe venissero fatti dei corsi di primo soccorso nelle scuole, sarebbero molto utili!",2022-11-14T21:58:05.000Z,13,la professionalità e tempestività di questa ragazza sono encomiabili bisognerebbe venissero fatti dei corsi di primo soccorso nelle scuole sarebbero molto utili,"[[la, professionalità, e, tempestività, di, questa, ragazza, sono, encomiabili, bisognerebbe, venissero, fatti, dei, corsi, di, primo, soccorso, nelle, scuole, sarebbero, molto, utili]]","[[la, professionalità, e, tempestività, di, questo, ragazza, essere, encomiabile, bisognare, venire, fatto, dio, corso, di, primo, soccorso, nella, scuola, essere, molto, utile]]","[[RD, S, CC, S, E, DD, S, V, A, V, VA, V, E_RD, S, E, NO, S, E_RD, S, V, B, A]]"
2,Ck9MRyyKIT5,user_0000030353,Nessuna correlazione con il siero? Fatevi domande gente! Troppi casi in giro!,2022-11-16T07:00:14.000Z,1,nessuna correlazione con il siero fatevi domande gente troppi casi in giro,"[[nessuna, correlazione, con, il, siero, fatevi, domande, gente, troppi, casi, in, giro]]","[[nessuno, correlazione, con, il, siero, fatevi, domanda, gente, troppo, caso, in, girare]]","[[DI, S, E, RD, S, V, S, S, DI, S, E, S]]"
3,Ck9MRyyKIT5,user_0000028606,Se stata l’angelo di quel l ‘uomo 👏🏻👏🏻👏🏻,2022-11-14T21:35:28.000Z,2,se stata langelo di quel l uomo,"[[se, stata, langelo, di, quel, l, uomo]]","[[se, stare, langelo, di, quello, l, uomo]]","[[CS, V, A, E, PD, RD, S]]"
4,Ck9MRyyKIT5,user_0000055719,Andrebbe insegnato a scuola,2022-11-15T05:08:19.000Z,1,andrebbe insegnato a scuola,"[[andrebbe, insegnato, a, scuola]]","[[andare, insegnare, a, scuola]]","[[VA, V, E, S]]"
...,...,...,...,...,...,...,...,...,...
206355,CiW8UfIMwND,user_0000056164,Mitica Oriana!😍,2022-09-11T08:33:12.000Z,0,mitica oriana,"[[mitica, oriana]]","[[mitico, oriana]]","[[A, A]]"
206356,CiW8UfIMwND,user_0000010176,🙏❤🙏,2022-09-11T08:13:04.000Z,0,,[],[],[]
206357,CiW8UfIMwND,user_0000028852,Esatto 😍,2022-09-11T08:12:32.000Z,0,esatto,[[esatto]],[[esatto]],[[A]]
206358,CiW8UfIMwND,user_0000033569,"Matteo, stai raschiando il fondo del barile?",2022-09-11T08:08:17.000Z,0,matteo stai raschiando il fondo del barile,"[[matteo, stai, raschiando, il, fondo, del, barile]]","[[matteo, stare, raschiare, il, fondere, del, barile]]","[[S, VA, V, RD, S, E_RD, S]]"


# Word2Vec

## Utils

In [4]:
import scipy as scipy

from gensim.models import Word2Vec, LdaModel
from gensim.models.phrases import Phrases, Phraser
from gensim.corpora import Dictionary

"""
Word 2 Vec
"""

def w2v(sentences, model='SG', vector_size=300, min_count=10, get_bigrams=True, min_count_bigrams=10):
    if get_bigrams:
        sentences = generate_bi_grams(sentences, min_count=min_count_bigrams, threshold=0.5, progress_per=1000, scoring='npmi')

    w2v_model = Word2Vec(sentences, min_count=min_count, vector_size=vector_size, sg=(0 if model=='CBOW' else 1))
    return w2v_model

def get_w2v_vocab(w2v_model):
    return w2v_model.wv.key_to_index

def generate_bi_grams(sentences, **kwargs):
    phrases = Phrases(sentences, **kwargs)
    phrases_model = Phraser(phrases)
    return [phrases_model[sent] for sent in sentences]

# --------------------------------------------------------------------------- #

""" Word embeddings utils """

def wvec(wv_model, w):
    return wv_model.wv[w]

def wnorm(wv_model, w):
    vec = wv_model.wv[w] if type(w) is str else w
    return math.sqrt(sum([math.pow(v,2) for v in vec]))

def wsim(wv_model, pos, neg=[], topn=10, thresh=0, comp=-1):
    pos = [pos] if type(pos) is str else pos
    neg = [neg] if type(neg) is str else neg
    sim0 = list(wv_model.wv.most_similar(positive=pos, negative=neg, topn=topn))
    if thresh > 0:
        sim0 = [s for s in sim0 if s[1] >= thresh]
    if comp != -1:
        sim0 = [s[0] if comp==0 else s[1] for s in sim0]
    return sim0

def vsim(wv_model, vector, topn=10, comp=-1):
    sim = wv_model.wv.similar_by_vector(vector, topn=topn)
    if comp == 0:
        return [s[0] for s in sim]
    elif comp == 1:
        return [s[1] for s in sim]  
    else:
        return sim

def wanal(wv_model, str_anal, topn=10, thresh=0):
    p1 = str_anal[:str_anal.index('=')]
    p2 = str_anal[str_anal.index('=')+1:]
    neg = p1[:p1.index(':')].replace(' ', '')
    pos1 = p1[p1.index(':')+1:].replace(' ', '')
    pos2 = p2[:p2.index(':')].replace(' ', '')
    return wsim(wv_model, [pos1, pos2], neg, topn=topn, thresh=thresh)

def wdist(wv_model, w1, w2):
    w1 = wv_model.wv[w1] if type(w1) is str else w1
    w2 = wv_model.wv[w2] if type(w2) is str else w2
    return 1-scipy.spatial.distance.cosine(w1, w2)

def nwdist(wv_model, ws1, ws2):
    ws1 = [w for w in ws1 if w in wv_model.wv.key_to_index]
    ws2 = [w for w in ws2 if w in wv_model.wv.key_to_index]
    return wv_model.wv.n_similarity(ws1,ws2)

def wchoesion(wv_model, words):
    wsum = 0
    for w1 in words:
        for w2 in words:
            wsum = wsum + ((1-wdist(wv_model, w1, w2)) if w1!=w2 else 0)
    return (wsum / (math.pow(len(words),2)-len(words))) if (math.pow(len(words),2)-len(words))!=0 else 0

# Build model

In [5]:
# w2v needs list of sentences: list[list[string]]

sentences = [sentence for comment in df.comment_lemma for sentence in comment if sentence != [] and sentence!=None]

In [6]:
w2v_salvini = w2v(sentences, model='SG', vector_size=150, min_count=5, get_bigrams=True)
w2v_salvini_vocab = get_w2v_vocab(w2v_salvini)

collecting all words and their counts
PROGRESS: at sentence #0, processed 0 words and 0 word types
PROGRESS: at sentence #1000, processed 6496 words and 5839 word types
PROGRESS: at sentence #2000, processed 14202 words and 11309 word types
PROGRESS: at sentence #3000, processed 21540 words and 15855 word types
PROGRESS: at sentence #4000, processed 30380 words and 20984 word types
PROGRESS: at sentence #5000, processed 38575 words and 25496 word types
PROGRESS: at sentence #6000, processed 46117 words and 29603 word types
PROGRESS: at sentence #7000, processed 53913 words and 33158 word types
PROGRESS: at sentence #8000, processed 62003 words and 37024 word types
PROGRESS: at sentence #9000, processed 68729 words and 40099 word types
PROGRESS: at sentence #10000, processed 76176 words and 43386 word types
PROGRESS: at sentence #11000, processed 84821 words and 47217 word types
PROGRESS: at sentence #12000, processed 91576 words and 49886 word types
PROGRESS: at sentence #13000, proces

In [21]:
wsim(w2v_salvini, 'magistratura', topn=100)

[('dittatura', 0.8013535737991333),
 ('fomentare', 0.7936769127845764),
 ('direttivo', 0.791358470916748),
 ('organizzazione', 0.771776556968689),
 ('soros', 0.7713298797607422),
 ('lobby', 0.7678341269493103),
 ('comunità_europeo', 0.7638823390007019),
 ('casto', 0.7624161839485168),
 ('toga', 0.7579337954521179),
 ('giornalaio', 0.7567523121833801),
 ('rovinare_dellitalia', 0.7558698058128357),
 ('multinazionale', 0.755838930606842),
 ('strategia', 0.7520187497138977),
 ('corrotto', 0.7516555786132812),
 ('burocrate', 0.7489603161811829),
 ('proprietà_privato', 0.7483412027359009),
 ('europeista', 0.7471895813941956),
 ('legemonia', 0.7462050914764404),
 ('ricatto', 0.745956540107727),
 ('minaccia', 0.7453929781913757),
 ('gestione', 0.7453019022941589),
 ('raggio', 0.7446823716163635),
 ('complicità', 0.7434127330780029),
 ('massone', 0.7427975535392761),
 ('scomparire', 0.7418749928474426),
 ('maggioranza', 0.7407003045082092),
 ('acqua_territoriale', 0.7394383549690247),
 ('uscire